In [1]:
% matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

from datetime import datetime, date, time
from textblob import TextBlob
from pre_processing import *
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\Andres
[nltk_data]     Montero\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
DATA_FOLDER = './data'

tweets_col_names=['text']

tweets_dtypes = {'text': str }


In [3]:
tweets_neg = pd.read_fwf(DATA_FOLDER + '/train_neg.txt',  names=tweets_col_names,
                              dtype=tweets_dtypes)
tweets_neg.head(20)

,text
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...
1,glad i dot have taks tomorrow ! ! #thankful #s...
2,1-3 vs celtics in the regular season = were fu...
3,<user> i could actually kill that girl i'm so ...
4,<user> <user> <user> i find that very hard to ...
5,wish i could be out all night tonight ! <user>
6,<user> i got kicked out the wgm
7,rt <user> <user> <user> yes she is ! u tell it...
8,why is she so perfect <url>
9,<user> hi harry ! did u havea good time in aus...


In [4]:
tweets_neg.drop_duplicates(inplace=True)

In [5]:
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: filter_digits(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: separate_hash(x))
tweets_neg['text'] = tweets_neg['text'].str.replace('#', '')
tweets_neg['text'] = tweets_neg['text'].str.replace('ur', 'your')
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: expand_contractions(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: interpret_emoji(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: remove_punctuation(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: remove_words(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: replace_moreletters(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: split_number_text(x))

tweets_neg.head()

,text
0,vinco tresorpack difficulty of object disassem...
1,glad i dot have taks tomorrow thankful startho
2,vs celtics in the regular season were fucked i...
3,i could actually kill that girl i am so sorry
4,i find that very hard to believe im afraid


In [6]:
stop = stopwords.words('english')
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
tweets_neg['text'].head()

0    vinco tresorpack difficulty object disassemble...
1              glad dot taks tomorrow thankful startho
2       vs celtics regular season fucked play playoffs
3                       could actually kill girl sorry
4                          find hard believe im afraid
Name: text, dtype: object

### Remove more comon words of the data set

In [7]:
freq_max = pd.Series(' '.join(tweets_neg['text']).split()).value_counts()[:10]
freq_max

yourl     28483
frame      6690
rt         4225
like       4093
get        3756
yoyour     3698
go         3662
one        3336
want       3259
miss       3110
dtype: int64

In [8]:
freq_max = list(freq_max.index)
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max))
tweets_neg['text'].head()

0    vinco tresorpack difficulty object disassemble...
1              glad dot taks tomorrow thankful startho
2       vs celtics regular season fucked play playoffs
3                       could actually kill girl sorry
4                          find hard believe im afraid
Name: text, dtype: object

In [9]:
freq_min = pd.Series(' '.join(tweets_neg['text']).split()).value_counts()[-10:]
freq_min

thankkss         1
videogames       1
markinch         1
crawler          1
tyouring's       1
cbawiththis      1
hscr             1
mistakenly       1
installations    1
bessie's         1
dtype: int64

In [10]:
freq_min = list(freq_min.index)
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min))
tweets_neg['text'].head()

0    vinco tresorpack difficulty object disassemble...
1              glad dot taks tomorrow thankful startho
2       vs celtics regular season fucked play playoffs
3                       could actually kill girl sorry
4                          find hard believe im afraid
Name: text, dtype: object

In [11]:
test = tweets_neg
test

,text
0,vinco tresorpack difficulty object disassemble...
1,glad dot taks tomorrow thankful startho
2,vs celtics regular season fucked play playoffs
3,could actually kill girl sorry
4,find hard believe im afraid
5,wish could night tonight
6,got kicked wgm
7,yes tell lips closed okay
8,perfect
9,hi harry havea good time aus didnt see maybe n...


In [12]:
# test['text'] = test['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
# test

In [13]:
 #Apply all preprocessing functions from preprocessing file
        #Filter digits
        line=filter_digits(line)
        #Remove Stop words
        line=remove_stopwords(line)
        #Interpret emojis in Tweet
        line=interpret_emoji(line)
        #Remove punctuation
        line=remove_punctuation(line)
        #Remove words user, url, number
        line=remove_words(line)
        #Remove repeated letters
        line=replace_moreletters(line)
        #Lematize words in the tweet
        line=lemmatize(line)
        #Split number and text in tweets
        line=split_number_text(line)
        #Correct misspell in tweet
        line=correct_misspell(line)
        

IndentationError: unexpected indent (<ipython-input-13-9df9e5126130>, line 3)